In [1]:
import sys
!conda install --yes --prefix {sys.prefix} pandas

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\User\anaconda3\envs\kate_ds

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.2.1   |       haa95532_0         123 KB
    ------------------------------------------------------------
                                           Total:         123 KB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> pkgs/main::ca-certificates-2022.2.1-haa95532_0
  openssl            conda-forge::openssl-1.1.1l-h8ffe710_0 --> pkgs/main::openssl-1.1.1m-h2bbff1b_0




ca-certificates-2022 | 123 KB    |            |   0% 
ca-certificates-2022 | 123 KB    | #3         |  13% 
ca-certificates-2022 | 123 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working...

In [3]:
import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv('201809-citibike-tripdata.csv')

In [10]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


1. Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов.

In [11]:
data.shape

(1877884, 15)

2. Найти среднюю длину поездок в минутах (столбец tripduration) c точностью до 2 знака.

In [12]:
data['tripduration'].apply(lambda x: x/60).mean().round(2)

16.13

3. Сколько поездок начались и закончились в той же самой станции?

In [14]:
data[(data['start station id'] == data['end station id'])].shape

(41364, 15)

4. Какой самый используемый байк (bikeid) в городе по количеству поездок?

In [15]:
data['bikeid'].value_counts().idxmax()

33875

5. Найдите идентификатор велосипеда (bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных.

In [17]:
data.groupby(['bikeid'])[['tripduration']].mean().idxmax()

tripduration    17548
dtype: int64

6. Найдите количество строк, в которых отсутствуют данные о start station id.

In [18]:
data[data['start station id'].isnull()].shape

(716, 15)

7. Какова средняя продолжительность поездки в минутах в зависимости от типа подписки c точностью до 2 знака?

In [19]:
data.groupby(['usertype'])['tripduration'].mean().apply(lambda x: x/60).round(2)

usertype
Customer      33.42
Subscriber    13.33
Name: tripduration, dtype: float64

8. Найдите среднюю длину поездок в километрах с точностью до 2 знака, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id).
Можно воспользоваться библиотекой geopy и взять расстояние geodesic(минимальное расстояние между точками).

In [21]:
conda install -c conda-forge geopy

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\User\anaconda3\envs\kate_ds

  added / updated specs:
    - geopy


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2022.2.1-h~ --> conda-forge::ca-certificates-2021.10.8-h5b45459_0
  openssl              pkgs/main::openssl-1.1.1m-h2bbff1b_0 --> conda-forge::openssl-1.1.1l-h8ffe710_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [24]:
import geopy
from geopy.distance import geodesic
from geopy import Point

In [25]:
data['point1'] = data.apply(lambda row: Point(latitude=row['start station latitude'], longitude=row['start station longitude']), axis=1)
data['point2'] = data.apply(lambda row: Point(latitude=row['end station latitude'], longitude=row['end station longitude']), axis=1)
data['distance_km'] = data.apply(lambda row: geodesic(row['point1'], row['point2']).km, axis=1)

In [26]:
data['distance_km'].mean().round(2)

1.81

9. Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера включительно.

In [27]:
times = pd.to_datetime(data.starttime)
data[times.dt.hour.isin([18, 19, 20])][['start station id']].value_counts().head()

start station id
519.0               3181
426.0               2701
402.0               2680
459.0               2605
477.0               2418
dtype: int64

10. Выберите идентификаторы станций (end station id), в которые приезжают с 6 до 10 утра включительно.

In [28]:
end_stations = [3140, 3106, 3116, 369]
data[(times.dt.hour.isin([6, 7, 8, 9, 10])) & (data['end station id'].isin(end_stations))]['end station id'].unique()

array([3116., 3140.,  369., 3106.])